In [213]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np

import pygame
from pygame.locals import *
import pygame.gfxdraw


In [214]:
with open("/Users/ard/Desktop/Coding_2/F1_laptime_project/f1-circuits.geojson") as f:
    gj = json.load(f)

In [215]:
melborne_data = gj["features"][0]["geometry"]["coordinates"]
scaler = 10

# High-resolution logic surface (you draw everything here)
SWIDTH, SHEIGHT = 1280, 960

WIDTH, HEIGHT = SWIDTH*scaler, SHEIGHT*scaler

offset = 1000

screen_width = WIDTH - offset
screen_height = HEIGHT - offset

x = []
y = []

for i in melborne_data:
    lon = i[0]
    lat = i[1]
    x.append(lon * 111.320)
    y.append(lat * 110.574)

range_x = max(x) - min(x)
range_y = max(y) - min(y)

if range_x / screen_width > range_y / screen_height:
    scale = screen_width / range_x
else:
    scale = screen_height / range_y

height = range_y * scale
circuit_data = []

for i in range(len(x)):
    screen_x = (x[i] - min(x)) * scale + offset/2 
    screen_y = height - (y[i] - min(y)) * scale + offset/2   
    circuit_data.append((screen_x, screen_y))

In [216]:
from scipy.interpolate import splprep, splev

x, y = zip(*circuit_data)
tck, u = splprep([x, y], s=30.0,per=True)
u_new = np.linspace(0, 1, 5000)
x_smooth, y_smooth = splev(u_new, tck)
path = np.stack((x_smooth, y_smooth), axis=1)

# 2. Generate thick track by computing offset vectors
def make_thick_polyline(path, width):
    left = []
    right = []
    for i in range(len(path) - 1):
        p1 = path[i]
        p2 = path[i + 1]
        dx, dy = p2 - p1
        length = np.hypot(dx, dy)
        if length == 0: continue
        nx, ny = -dy / length, dx / length  # perpendicular vector
        offset = np.array([nx, ny]) * (width / 2)
        left.append(p1 + offset)
        right.append(p1 - offset)
    # Add final point
    offset = np.array([-dy, dx]) / length * (width / 2)
    left.append(path[-1] + offset)
    right.append(path[-1] - offset)
    return left + right[::-1]  # combine as polygon

track_width = 500  # Change this to your desired width
track_polygon = make_thick_polyline(path, track_width)

In [ ]:

# Initialize pygame
pygame.init()

GAME_SURF = pygame.Surface((WIDTH, HEIGHT))

# Display resolution (smaller)
WINDOW = pygame.display.set_mode((SWIDTH, SHEIGHT))
pygame.display.set_caption("Downscaled Output")

FPS = 60
clock = pygame.time.Clock()

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)


running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    GAME_SURF.fill(WHITE)
    pygame.draw.polygon(GAME_SURF, BLACK, track_polygon)

    scaled_surface = pygame.transform.smoothscale(GAME_SURF, (SWIDTH, SHEIGHT))

    WINDOW.blit(scaled_surface, (0, 0))
    pygame.display.update()

    clock.tick(FPS)

pygame.quit()


KeyboardInterrupt: 

: 